In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import signal
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from glob import glob
import math
from sklearn.model_selection import train_test_split
from PIL.ImageOps import invert
from keras.utils import plot_model
from sklearn.metrics import classification_report
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint

np.random.seed(1234)
tf.set_random_seed(1234)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

K.clear_session()

from keras.backend.tensorflow_backend import set_session 
config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True 
# dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran) 
# (nothing gets printed in Jupyter, only if you run it standalone) 
sess = tf.Session(config=config)
set_session(sess) # set this TensorFlow session as the default session for Keras

In [ ]:
concrete_paths = glob('../train-set/normal/post/*_concrete_*')
metallic_paths = glob('../train-set/normal/post/*_pec_*')
plastic_paths = glob('../train-set/normal/post/*_pvc_*')
real_path = glob('/home/will/data/*.png')

def load_img(path):
    img = image.load_img(path, grayscale=True, target_size=(256,256), interpolation='hamming')
    #img = invert(img)
    return image.img_to_array(img)

concrete_scans = np.array([load_img(path) for path in concrete_paths])
metallic_scans = np.array([load_img(path) for path in metallic_paths])
plastic_scans = np.array([load_img(path) for path in plastic_paths])

concrete_labels = np.zeros((len(concrete_scans),))
metallic_labels = np.ones((len(metallic_scans),))
plastic_labels = np.ones((len(plastic_scans),)) * 2

y = np.concatenate([concrete_labels, metallic_labels, plastic_labels]).astype(np.int32)
a_scans = np.concatenate([concrete_scans, metallic_scans, plastic_scans])

print(y.shape)
print(a_scans.shape)

print(concrete_scans.shape)
print(metallic_scans.shape)
print(plastic_scans.shape)

In [ ]:
from keras.layers.merge import _Merge, multiply
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

GRADIENT_PENALTY_WEIGHT = 10 # As per the paper

class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])


class GPRGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 3
        self.latent_dim = 100
        self.losslog = []
        

        self.n_critic = 5
        optimizer = Adam(0.0003, 0.5)
        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        
        # Generate image based of noise (fake sample) and add label to the input 
        label = Input(shape=(1,))
        fake_img = self.generator([z_disc, label])

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        averaged_samples = RandomWeightedAverage()([real_img, fake_img])
        
        # Determine validity of weighted sample
        validity_interpolated = self.critic(averaged_samples)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=averaged_samples)
        # Functions need names or Keras will throw an error
        partial_gp_loss.__name__ = 'gradient_penalty'

        self.critic_model = Model(inputs=[real_img, label, z_disc],
                                  outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                        self.wasserstein_loss,
                                        partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        
        self.aux_model = self.build_classifier()
        
        real_pred = self.aux_model(real_img)
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # add label to the input
        #label = Input(shape=(1,))
        # Generate images based of noise
        img = self.generator([z_gen, label])
        # Discriminator determines validity
        valid = self.critic(img)
        #gen_pred = self.aux_model(img)
        # Defines generator model
        self.generator_model = Model([z_gen, label], [valid])
        self.generator_model.compile(loss=self.wasserstein_loss,
                                     optimizer=optimizer)
        
        
        self.aux_model.compile(loss='sparse_categorical_crossentropy',
                               optimizer='adam', metrics=['accuracy'])
        
        
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    
    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def build_generator(self):

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')

        out = Dense(128 * 8 * 8,
                        activation="relu",
                        input_dim=self.latent_dim)(noise)
        out = Reshape((8, 8, 128))(out)
        
        out = BatchNormalization(momentum=0.8)(out)
        out = UpSampling2D()(out)
        out = Conv2D(256, kernel_size=5, padding="same")(out)
        out = Activation("relu")(out)
        
        out = BatchNormalization(momentum=0.8)(out)
        out = UpSampling2D()(out)
        model.add(Conv2D(128, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(32, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(16, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        #model.summary()

        
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        gen = Model([noise, label], img)
        print(gen.summary())
        return gen

    def build_critic(self):

        model = Sequential()
        model.add(Conv2D(16, kernel_size=5, strides=2, padding="same", input_shape=self.img_shape))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.25))
        model.add(Conv2D(16*2, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.25))
        model.add(Conv2D(16*4, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.25))
        model.add(Conv2D(16*8, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.25))
        model.add(Conv2D(16*16, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.25))
        model.add(Flatten())
        #model.summary()

        img = Input(shape=self.img_shape)
        
        features = model(img)
        # Determine validity and label of the image
        validity = Dense(1, activation=None)(features)
        label_prob = Dense(self.num_classes, activation="softmax")(features)

        model = Model(img, validity)
        print(model.summary())
        return model
    
    def build_classifier(self):
        model = Sequential()
        model.add(Conv2D(16, kernel_size=5, strides=2, padding="same", input_shape=self.img_shape))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Conv2D(32, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Flatten())
        model.add(Dense(self.num_classes, activation='softmax'))
        img = Input(shape=self.img_shape)
        
        label = model(img)
        
        return Model(img, label)

    def train(self, epochs, batch_size=16, sample_interval=50):
        
        aux_checkpoint = ModelCheckpoint('checkpoints/aux_model.h5',
                                         save_best_only=True)
        generator_checkpoint = ModelCheckpoint('checkpoints/generator_freq.h5',
                                               save_best_only=True,
                                               save_weights_only=True)
        disc_checkpoint = ModelCheckpoint('checkpoints/disc_freq.h5',
                                          save_best_only=True)
        # Load the dataset
        X_train, X_test, y_train, y_test = train_test_split(a_scans, y, test_size=0.3)
        
        

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        y_train = y_train.reshape(-1, 1)
        y_test = y_test.reshape(-1, 1)
        
        np.save('X_test.npy', X_test)
        np.save('y_test.npy', y_test)

         # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        
        test_loss = []
            
        for epoch in tqdm(range(epochs+1)):
            
            for _ in range(self.n_critic):
                sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)

                # ---------------------
                #  Train Discriminator
                # ---------------------
                
                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                test_idx = np.random.randint(0, X_test.shape[0], batch_size)
                imgs, labels = X_train[idx], y_train[idx]
                test_imgs, test_labels = X_test[test_idx], y_test[test_idx]
                
                
                # Sample generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                # Train the critic
                d_loss = self.critic_model.fit([imgs, labels, noise],
                                               [valid, fake, dummy],
                                               batch_size=batch_size,
                                               validation_data=[[test_imgs, test_labels, noise],
                                                                [valid, fake, dummy]],
                                               verbose=0,
                                               callbacks=[disc_checkpoint])


            g_loss = self.generator_model.fit([noise, labels],
                                              [valid], 
                                              batch_size=batch_size,
                                              validation_data=[[noise, test_labels],
                                                               [valid]],
                                              verbose=0,
                                              callbacks=[generator_checkpoint])
            
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(f'generator_normal_{epoch}')

            with open('loss_same_labels.log', 'w') as f:
                f.writelines('d_loss, g_loss\n')
                for each in self.losslog:
                    f.writelines('%s, %s\n'%(each[0], each[1]))
                    
        
                        
    def sample_images(self, epoch):
        r, c = 4, 4
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([np.random.randint(0, 3, 1) for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.critic, "discriminator")
        
    def plot_loss(self, gen_loss, disc_loss):
        gen_loss = np.array(gen_loss)
        disc_loss = np.array(disc_loss)
        plt.plot(gen_loss)
        plt.savefig('saved_model/gen_loss.png')
        plt.plot(disc_loss)
        plt.savefig('saved_model/disc_loss.png')

In [ ]:
gpr_gan = GPRGAN()

gpr_gan.train(epochs=50000, batch_size=32, sample_interval=500)